SOLUTION: ARTIFICIAL NEURAL NETWORK WITH ONE HIDDEN LAYER:

In [20]:
# CODE implementing ANN for binary classification (with one hidden layer)

# Importing all the required libraries

import os
import numpy as np
import cv2
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from sklearn.impute import SimpleImputer


# Loading the training data
train_raveling = os.listdir('train/Raveling')
train_non_raveling = os.listdir('train/Non_raveling')
train_images = []
train_labels = []

# preprocess the training data
for img in train_raveling:
    image = cv2.imread('train/Raveling/' + img)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    h, w, _ = image.shape
    total_pixels = h * w
   
    # Calculating features as given in the paper 

    mean = np.mean(image, axis=(0, 1))
    std = np.std(image, axis=(0, 1))
    skewness = np.mean((image - mean) ** 3, axis=(0, 1)) / (std ** 3)
    kurtosis = np.mean((image - mean) ** 4, axis=(0, 1)) / (std ** 4) 
    entropy = -np.sum(np.log2(image / 256) * (image / 256), axis=(0, 1))
    rng = np.max(image, axis=(0, 1)) - np.min(image, axis=(0, 1))
    first_order_histogram = []
    total_pixels = image.shape[0] * image.shape[1]
    first_order_histogram = []
    for c in range(3):
        n_c = np.sum(image[:, :, c])
        Pc_I = n_c / total_pixels
        first_order_histogram.append(Pc_I)

    mean_mod = mean * first_order_histogram
    std_mod = std * first_order_histogram
    skewness_mod = skewness * first_order_histogram
    kurtosis_mod = kurtosis * first_order_histogram
    entropy_mod = entropy * first_order_histogram
    rng_mod = rng * first_order_histogram
    features = np.concatenate([mean_mod, std_mod, skewness_mod, kurtosis_mod, entropy_mod, rng_mod])
    train_images.append(features)
    train_labels.append(1)  #label raveling as 1

for img in train_non_raveling:
    image = cv2.imread('train/Non_raveling/' + img)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    h, w, _ = image.shape
    total_pixels = h * w

    # Calculating features as given in the paper 

    mean = np.mean(image, axis=(0, 1))
    std = np.std(image, axis=(0, 1))
    skewness = np.mean((image - mean) ** 3, axis=(0, 1)) / (std ** 3)
    kurtosis = np.mean((image - mean) ** 4, axis=(0, 1)) / (std ** 4) 
    entropy = -np.sum(np.log2(image / 256) * (image / 256), axis=(0, 1))
    rng = np.max(image, axis=(0, 1)) - np.min(image, axis=(0, 1))
    first_order_histogram = []
    total_pixels = image.shape[0] * image.shape[1]
    first_order_histogram = []
    for c in range(3):
        n_c = np.sum(image[:, :, c])
        Pc_I = n_c / total_pixels
        first_order_histogram.append(Pc_I)
    mean_mod = mean * first_order_histogram
    std_mod = std * first_order_histogram
    skewness_mod = skewness * first_order_histogram
    kurtosis_mod = kurtosis * first_order_histogram
    entropy_mod = entropy * first_order_histogram
    rng_mod = rng * first_order_histogram
    features = np.concatenate([mean_mod, std_mod, skewness_mod, kurtosis_mod, entropy_mod, rng_mod])
    train_images.append(features)
    train_labels.append(0)  #label non-raveling as 0
train_images = np.array(train_images)
train_labels = np.array(train_labels) 

# removing NaN error
imp = SimpleImputer(strategy='mean')
train_images = imp.fit_transform(train_images)
scaler = StandardScaler()
train_images = scaler.fit_transform(train_images)


# Building a shallow neural network with a single hidden layer (128 units)

# model = keras.Sequential([keras.layers.InputLayer(input_shape=(train_images.shape[1],)),
#     keras.layers.Dense(128, activation='relu'),
#     keras.layers.Dense(1, activation='sigmoid')])   0.9666 score

model = keras.Sequential([keras.layers.InputLayer(input_shape=(train_images.shape[1],)),
    keras.layers.Dense(128, activation='relu'),keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')])

# Compile the model with binary cross-entropy loss, Adam optimizer, and accuracy metric
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the model to the training data
model.fit(train_images, train_labels, epochs=350, batch_size=32)


test_images=[]
filenames=[]

# Loading  and preprocessing the test data

for i in range(1, 301):
    image = cv2.imread('test/' + str(i) + '.jpg')
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    h, w, _ = image.shape
    total_pixels = h * w
    mean = np.mean(image, axis=(0, 1))
    std = np.std(image, axis=(0, 1))
    skewness = np.mean((image - mean) ** 3, axis=(0, 1)) / (std ** 3)
    kurtosis = np.mean((image - mean) ** 4, axis=(0, 1)) / (std ** 4) 
    entropy = -np.sum(np.log2(image / 256) * (image / 256), axis=(0, 1))
    rng = np.max(image, axis=(0, 1)) - np.min(image, axis=(0, 1))
    first_order_histogram = []
    total_pixels = image.shape[0] * image.shape[1]
    first_order_histogram = []
    for c in range(3):
        n_c = np.sum(image[:, :, c])
        Pc_I = n_c / total_pixels
        first_order_histogram.append(Pc_I)

    mean_mod = mean * first_order_histogram
    std_mod = std * first_order_histogram
    skewness_mod = skewness * first_order_histogram
    kurtosis_mod = kurtosis * first_order_histogram
    entropy_mod = entropy * first_order_histogram
    rng_mod = rng * first_order_histogram
    features = np.concatenate([mean_mod, std_mod, skewness_mod, kurtosis_mod, entropy_mod, rng_mod])
    test_images.append(features)
    filenames.append(str(i) + '.jpg')

test_images = np.array(test_images)
imp = SimpleImputer(strategy='mean')
test_images = imp.fit_transform(test_images)
test_images = scaler.transform(test_images)
predictions=[]
# Predicting the test labels
predictions = model.predict(test_images)


C:\Users\HP\AppData\Local\Temp\ipykernel_9004\2242147268.py:32: RuntimeWarning: divide by zero encountered in log2
  entropy = -np.sum(np.log2(image / 256) * (image / 256), axis=(0, 1))
C:\Users\HP\AppData\Local\Temp\ipykernel_9004\2242147268.py:32: RuntimeWarning: invalid value encountered in multiply
  entropy = -np.sum(np.log2(image / 256) * (image / 256), axis=(0, 1))


Epoch 1/350
22/22 [==============================] - 1s 2ms/step - loss: 0.5750 - accuracy: 0.7086
Epoch 2/350
22/22 [==============================] - 0s 2ms/step - loss: 0.4572 - accuracy: 0.7700
Epoch 3/350
22/22 [==============================] - 0s 2ms/step - loss: 0.4214 - accuracy: 0.8043
Epoch 4/350
22/22 [==============================] - 0s 2ms/step - loss: 0.3986 - accuracy: 0.8229
Epoch 5/350
22/22 [==============================] - 0s 3ms/step - loss: 0.3819 - accuracy: 0.8314
Epoch 6/350
22/22 [==============================] - 0s 3ms/step - loss: 0.3649 - accuracy: 0.8514
Epoch 7/350
22/22 [==============================] - 0s 3ms/step - loss: 0.3486 - accuracy: 0.8514
Epoch 8/350
22/22 [==============================] - 0s 2ms/step - loss: 0.3342 - accuracy: 0.8629
Epoch 9/350
22/22 [==============================] - 0s 2ms/step - loss: 0.3238 - accuracy: 0.8614
Epoch 10/350
22/22 [==============================] - 0s 3ms/step - loss: 0.3123 - accuracy: 0.8629
Epoch 11/

22/22 [==============================] - 0s 3ms/step - loss: 0.0643 - accuracy: 0.9757
Epoch 84/350
22/22 [==============================] - 0s 3ms/step - loss: 0.0634 - accuracy: 0.9843
Epoch 85/350
22/22 [==============================] - 0s 3ms/step - loss: 0.0651 - accuracy: 0.9800
Epoch 86/350
22/22 [==============================] - 0s 2ms/step - loss: 0.0622 - accuracy: 0.9800
Epoch 87/350
22/22 [==============================] - 0s 3ms/step - loss: 0.0633 - accuracy: 0.9786
Epoch 88/350
22/22 [==============================] - 0s 3ms/step - loss: 0.0613 - accuracy: 0.9800
Epoch 89/350
22/22 [==============================] - 0s 3ms/step - loss: 0.0639 - accuracy: 0.9800
Epoch 90/350
22/22 [==============================] - 0s 3ms/step - loss: 0.0620 - accuracy: 0.9829
Epoch 91/350
22/22 [==============================] - 0s 3ms/step - loss: 0.0588 - accuracy: 0.9829
Epoch 92/350
22/22 [==============================] - 0s 3ms/step - loss: 0.0562 - accuracy: 0.9814
Epoch 93/350


22/22 [==============================] - 0s 2ms/step - loss: 0.0044 - accuracy: 1.0000
Epoch 246/350
22/22 [==============================] - 0s 2ms/step - loss: 0.0047 - accuracy: 1.0000
Epoch 247/350
22/22 [==============================] - 0s 2ms/step - loss: 0.0041 - accuracy: 1.0000
Epoch 248/350
22/22 [==============================] - 0s 2ms/step - loss: 0.0040 - accuracy: 1.0000
Epoch 249/350
22/22 [==============================] - 0s 2ms/step - loss: 0.0044 - accuracy: 1.0000
Epoch 250/350
22/22 [==============================] - 0s 2ms/step - loss: 0.0046 - accuracy: 1.0000
Epoch 251/350
22/22 [==============================] - 0s 2ms/step - loss: 0.0043 - accuracy: 1.0000
Epoch 252/350
22/22 [==============================] - 0s 2ms/step - loss: 0.0040 - accuracy: 1.0000
Epoch 253/350
22/22 [==============================] - 0s 2ms/step - loss: 0.0061 - accuracy: 1.0000
Epoch 254/350
22/22 [==============================] - 0s 2ms/step - loss: 0.0047 - accuracy: 1.0000
Epoc

C:\Users\HP\AppData\Local\Temp\ipykernel_9004\2242147268.py:121: RuntimeWarning: divide by zero encountered in log2
  entropy = -np.sum(np.log2(image / 256) * (image / 256), axis=(0, 1))
C:\Users\HP\AppData\Local\Temp\ipykernel_9004\2242147268.py:121: RuntimeWarning: invalid value encountered in multiply
  entropy = -np.sum(np.log2(image / 256) * (image / 256), axis=(0, 1))


10/10 [==============================] - 0s 3ms/step


In [21]:
print("Accuracy of model on training data",model.evaluate(train_images,train_labels)[1])



22/22 [==============================] - 0s 2ms/step - loss: 0.0017 - accuracy: 1.0000
Accuracy of model on training data 1.0


In [22]:
import pandas as pd
test_labels=[]
for i in range(0,len(predictions)):
    # Convert predictions above 0.5 to non-raveling and rest to raveling class     
    if predictions[i]<0.5:
        test_labels.append("Non_raveling")
    else:
        test_labels.append("Raveling")
#Storing filename and labels in dataframe     
df = pd.DataFrame({'filename': filenames, 'class': test_labels})
#Changing dataframe to csv file
df.to_csv('hzpredictionsAnn22103025Final.csv', index=False)    
######################################END OF CODE#######################################################

SOLUTION: LOGISTIC REGRESSION MODEL

In [26]:
# CODE implementing LOGISTIC REGRESSION for binary classification

# Importing all the required libraries


import pandas as pd
import os
import numpy as np
import cv2
from sklearn.linear_model import LogisticRegression
# Load and preprocess the training data
train_raveling = os.listdir('train/Raveling')
train_non_raveling = os.listdir('train/Non_raveling')
train_images = []
train_labels = []

for img in train_raveling:
    image = cv2.imread('train/Raveling/' + img)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    h, w, _ = image.shape
    total_pixels = h * w
    mean = np.mean(image, axis=(0, 1))
    std = np.std(image, axis=(0, 1))
    skewness = np.mean((image - mean) ** 3, axis=(0, 1)) / (std ** 3)
    kurtosis = np.mean((image - mean) ** 4, axis=(0, 1)) / (std ** 4) 
    entropy = -np.sum(np.log2(image / 256) * (image / 256), axis=(0, 1))
    rng = np.max(image, axis=(0, 1)) - np.min(image, axis=(0, 1))
    first_order_histogram = []
    total_pixels = image.shape[0] * image.shape[1]
    # Calculating features as given in the paper 
    
    first_order_histogram = []
    for c in range(3):
        n_c = np.sum(image[:, :, c])
        Pc_I = n_c / total_pixels
        first_order_histogram.append(Pc_I)

    mean_mod = mean * first_order_histogram
    std_mod = std * first_order_histogram
    skewness_mod = skewness * first_order_histogram
    kurtosis_mod = kurtosis * first_order_histogram
    entropy_mod = entropy * first_order_histogram
    rng_mod = rng * first_order_histogram
    features = np.concatenate([mean_mod, std_mod, skewness_mod, kurtosis_mod, entropy_mod, rng_mod])
    train_images.append(features)
    train_labels.append(1)  #label raveling as 1

for img in train_non_raveling:
    image = cv2.imread('train/Non_raveling/' + img)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    h, w, _ = image.shape
    total_pixels = h * w
    mean = np.mean(image, axis=(0, 1))
    std = np.std(image, axis=(0, 1))
    skewness = np.mean((image - mean) ** 3, axis=(0, 1)) / (std ** 3)
    kurtosis = np.mean((image - mean) ** 4, axis=(0, 1)) / (std ** 4) 
    entropy = -np.sum(np.log2(image / 256) * (image / 256), axis=(0, 1))
    rng = np.max(image, axis=(0, 1)) - np.min(image, axis=(0, 1))
    first_order_histogram = []
    total_pixels = image.shape[0] * image.shape[1]
    first_order_histogram = []
    for c in range(3):
        n_c = np.sum(image[:, :, c])
        Pc_I = n_c / total_pixels
        first_order_histogram.append(Pc_I)

    mean_mod = mean * first_order_histogram
    std_mod = std * first_order_histogram
    skewness_mod = skewness * first_order_histogram
    kurtosis_mod = kurtosis * first_order_histogram
    entropy_mod = entropy * first_order_histogram
    rng_mod = rng * first_order_histogram
    features = np.concatenate([mean_mod, std_mod, skewness_mod, kurtosis_mod, entropy_mod, rng_mod])
    train_images.append(features)
    train_labels.append(0)  #label non-raveling as 0
train_images = np.array(train_images)
train_labels = np.array(train_labels) 
import numpy as np
from sklearn.impute import SimpleImputer

# removing NaN error
imp = SimpleImputer(strategy='mean')
train_images = imp.fit_transform(train_images)
# Train the Logistic Regression model
model = LogisticRegression(max_iter=1000)
model.fit(train_images, train_labels)
#reading test images:
test_images=[]
filenames=[]
for i in range(1, 301):
    image = cv2.imread('test/' + str(i) + '.jpg')
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    h, w, _ = image.shape
    total_pixels = h * w
    mean = np.mean(image, axis=(0, 1))
    std = np.std(image, axis=(0, 1))
    skewness = np.mean((image - mean) ** 3, axis=(0, 1)) / (std ** 3)
    kurtosis = np.mean((image - mean) ** 4, axis=(0, 1)) / (std ** 4) 
    entropy = -np.sum(np.log2(image / 256) * (image / 256), axis=(0, 1))
    rng = np.max(image, axis=(0, 1)) - np.min(image, axis=(0, 1))
    first_order_histogram = []
    total_pixels = image.shape[0] * image.shape[1]
    first_order_histogram = []
    for c in range(3):
        n_c = np.sum(image[:, :, c])
        Pc_I = n_c / total_pixels
        first_order_histogram.append(Pc_I)

    mean_mod = mean * first_order_histogram
    std_mod = std * first_order_histogram
    skewness_mod = skewness * first_order_histogram
    kurtosis_mod = kurtosis * first_order_histogram
    entropy_mod = entropy * first_order_histogram
    rng_mod = rng * first_order_histogram
    features = np.concatenate([mean_mod, std_mod, skewness_mod, kurtosis_mod, entropy_mod, rng_mod])
    test_images.append(features)
    filenames.append(str(i) + '.jpg')
test_labels2=[]
test_images = np.array(test_images)
test_images=imp.fit_transform(test_images)
#predicting class of test image using our developed model:
test_labels = model.predict(test_images)
for i in range(len(test_labels)):
    if test_labels[i]==0:
        test_labels2.append("Non_raveling")
# #         test_labels2[i]="Non_raveling"
    elif test_labels[i]==1:
        test_labels2.append("Raveling")
#Checking order of accuracy of train data:
accuracy = model.score(train_images, train_labels)
print("Accuracy on training data:", accuracy)
df = pd.DataFrame({'filename': filenames, 'class': test_labels2})
#Changing dataframe to csv file
df.to_csv('predictionsLR1.csv', index=False)

######################################END OF CODE#######################################################

C:\Users\HP\AppData\Local\Temp\ipykernel_9004\1123209733.py:26: RuntimeWarning: divide by zero encountered in log2
  entropy = -np.sum(np.log2(image / 256) * (image / 256), axis=(0, 1))
C:\Users\HP\AppData\Local\Temp\ipykernel_9004\1123209733.py:26: RuntimeWarning: invalid value encountered in multiply
  entropy = -np.sum(np.log2(image / 256) * (image / 256), axis=(0, 1))
D:\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\HP\AppData\Local\Temp\ipykernel_9004\1123209733.py:99: RuntimeWarning: divide by zero encountered in log2
  entrop

Accuracy on training data: 0.8014285714285714
